# Saving and Exporting Data

This lesson covers:

* Saving and reloading data

This first block loads the data that was used in the previous lesson.

In [1]:
# Setup: Load the data to use later
import pandas as pd

gs10_csv = pd.read_csv("data/GS10.csv", index_col="DATE", parse_dates=True)
gs10_excel = pd.read_excel("data/GS10.xls", skiprows=10,
                           index_col="observation_date")


## Problem: Export to Excel

Export `gs10_csv` to the Excel file `gs10-exported.xlsx`.


In [2]:
gs10_csv.to_excel("gs10-exported.xlsx")


## Problem: Export to CSV

Export `gs10_excel` to CSV. 

In [3]:
gs10_csv.to_csv("gs10-exported.csv")

## Problem: Export to HDF

Export both to a single HDF file (the closest thing to a "native" format in pandas).

In [4]:
# mode="w" creates a new file for writing
gs10_csv.to_hdf("gs10.h5", "csv", mode="w")
# "a" allows an existing file to be appended to
gs10_excel.to_hdf("gs10.h5", "excel", mode="a")

## Problem: Import from HDF 

Import the data saved as HDF and verify it is the same as the original data.

In [5]:
gs10_csv_reloaded = pd.read_hdf("gs10.h5", "csv")
gs10_csv_reloaded.head()

,GS10
DATE,
1953-04-01,2.83
1953-05-01,3.05
1953-06-01,3.11
1953-07-01,2.93
1953-08-01,2.95


In [6]:
gs10_excel_reloaded = pd.read_hdf("gs10.h5", "excel")
gs10_excel_reloaded.head()


,GS10
observation_date,
1953-04-01,2.83
1953-05-01,3.05
1953-06-01,3.11
1953-07-01,2.93
1953-08-01,2.95
